# Tutorial7: qwen2-72b-int4 推理

本节旨在展示如何在 scow 平台上使用单机单卡运行 [Qwen2-72B-Instruct-GPTQ-Int4](https://huggingface.co/Qwen/Qwen2-72B-Instruct-GPTQ-Int4) 模型的推理任务。

分以下几步来实现：
1. 环境安装与应用创建
2. 下载模型
3. 模型推理


Qwen 系列模型是由阿里巴巴开发的。Qwen 模型系列包括不同规模的模型，参数范围从 0.5 到 720 亿，适用于各种应用场景，如文本生成、翻译、问答等。Qwen2-72B-Instruct-GPTQ-Int4 支持高达 131,072 个 token 的上下文长度，能够处理大量输入。

## 1. 环境安装与应用创建

首先在data shell中创建conda环境:
```bash
conda create -n tutorial7 python=3.9
conda activate tutorial7
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
pip install notebook jupyterlab accelerate numpy==1.26.4 matplotlib==3.8.4 optimum>1.13.2 auto-gptq>0.4.2 transformers>=4.32.0,<4.38.0 accelerate tiktoken einops transformers_stream_generator==0.0.4 scipy
pip install --upgrade pyarrow
```

（ pytorch 版本需与 cuda 版本对应，请查看版本对应网站：https://pytorch.org/get-started/previous-versions ）
（ 通过 nvidia-smi 命令可查看 cuda 版本）

然后创建JupyterLab应用, `Conda环境名`请填写`tutorial7`, 硬件资源为1个GPU，建议使用 A100 40G 或者 V100 32G。创建应用后, 进入应用并打开本文件。

CUDA Version: 12.1; Torch Version: 2.3.1

## 2. 下载模型

在data shell 中执行，命令执行位置在当前文件所在的文件夹。

```bash
# 如果以下目录存在， 可以直接复制:
cp -r /lustre/public/tutorial/models/models--Qwen--Qwen2-72B-Instruct-GPTQ-Int4/ ./

# 否则请下载:
export HF_ENDPOINT=https://hf-mirror.com
huggingface-cli download --resume-download Qwen/Qwen2-72B-Instruct-GPTQ-Int4 --local-dir models--Qwen--Qwen2-72B-Instruct-GPTQ-Int4
```

## 3. 模型推理

[[参考链接]](https://huggingface.co/Qwen/Qwen2-72B-Instruct-GPTQ-Int4)

运行下方代码进行模型推理:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 使用 GPU
device = "cuda"

# 模型路径
# VAR_PLACEHOLDER
model_path = "models--Qwen--Qwen2-72B-Instruct-GPTQ-Int4"

# 加载模型和分词器
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# prompt
prompt = "什么是大语言模型"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

# 生成回答
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)


推理过程中使用 nvidia-smi 命令可以查看 GPU 运行情况。

---

> 作者: 黎颖; 龙汀汀
>
> 联系方式: yingliclaire@pku.edu.cn;   l.tingting@pku.edu.cn